In [285]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('ggplot')
%matplotlib inline
from tqdm import tqdm
from multitest import MultiTest
import pandas as pd

from src.DetectLM import DetectLM
from src.fit_survival_function import fit_per_length_survival_function

HERE (8/7/2023): 
- There is some issue with the dataset 'wiki-long': the intersection between mahcine and human ids is very small. 
- It is unclear what is the cause. 
- Also, right now we sample a fraction of the human sentences but we need to sample a fraction of the machine sentences. 
- Therefore, we should specify the number of sentences to sample from the human text based on the number of sentences in the machine text. 


In [286]:
dataset_name = 'wiki-long'
model_name = "gpt2-xl"

params = {}
params['ignore-first-sentence'] = True
params['null-data-file'] = f"results/{model_name}_no_context_{dataset_name}_machine.csv"
params['language-model-name'] = model_name
params['number-of-interpolation-points'] = 47
params['max-tokens-per-sentence'] = 50
params['min-tokens-per-sentence'] = 8
params['hc-type'] = "stbl"

In [287]:
def get_null_data(params):
    df_null = pd.read_csv(params['null-data-file'])
    if params['ignore-first-sentence']: 
        df_null = df_null[df_null.num > 1]
    return df_null

def get_survival_function(df, G=101):
    """
    One survival function for every sentence length in tokens

    Args:
    :df:  data frame with columns 'response' and 'length'

    Return:
        bivariate function (length, responce) -> (0,1)

    """
    assert not df.empty
    value_name = "response" if "response" in df.columns else "logloss"

    df1 = df[~df[value_name].isna()]
    ll = df1['length']
    xx1 = df1[value_name]
    return fit_per_length_survival_function(ll, xx1, log_space=True, G=G)

       
df_null = get_null_data(params)
pval_functions = get_survival_function(df_null, G=params['number-of-interpolation-points'])

In [288]:
ds_machine = pd.read_csv(f"results/{model_name}_no_context_{dataset_name}_machine.csv")
ds_human = pd.read_csv(f"results/{model_name}_no_context_{dataset_name}_human.csv")

In [290]:
print(len(ds_machine))
print(len(ds_human))

117746
361391


In [284]:
ds_human[ds_human['name'] == 26709147]

,Unnamed: 0,num,length,response,context_length,name
0,0,1,45,2.818405,0,26709147
1,1,2,25,5.043517,0,26709147
2,2,3,14,3.004730,0,26709147
3,3,4,40,3.169291,0,26709147
4,4,5,46,4.241125,0,26709147
5,5,6,26,4.387065,0,26709147
6,6,7,16,3.758419,0,26709147
7,7,8,27,4.275131,0,26709147
8,8,9,16,3.509687,0,26709147
9,9,10,20,3.944451,0,26709147


In [291]:
eps = 0.1

ds_merged = ds_machine.merge(ds_human, on='name', how='inner')

joint_names = ds_merged['name'].tolist()


ds_pool = ds_human[ds_human['name'].isin(joint_names)]
ds_sample = ds_pool.groupby("name").sample(frac=eps)

print(len(ds_sample))

ds_sample['human'] = True
ds_machine['human'] = False
ds_mixed = pd.concat([ds_machine[ds_machine['name'].isin(joint_names)], ds_sample])

print(ds_mixed.groupby('name')['human'].mean().mean())

9780
0.07752279215394678


In [262]:
def group_articles_to_minimum_length(df, min_length):
    """
    Rearrange group names so that every group has at least
    :min_length: elements
    """
    df_grouped = df.copy()
    df_grouped.loc[:, 'new_name'] = df['name'].copy()
    groups = list(ds_mixed.groupby('name'))
    lo_names = []
    while len(groups) > 0:
        c = groups.pop(0)
        acc = len(c[1])
        while (acc <= min_length) and len(groups)>0:
            c1 = groups.pop(0)
            acc += len(c1[1])
            df_grouped.loc[df['name'] == c1[0], 'new_name'] = c[0]

    return df_grouped

min_length = 100
ds_mixed_grouped = group_articles_to_minimum_length(ds_mixed, min_length)


In [263]:
detectlm = DetectLM(lambda x: 0,
                    pval_functions,
                     min_len=params['min-tokens-per-sentence'],
                    max_len=params['max-tokens-per-sentence'],
                    HC_type=params['hc-type'],
                    ignore_first_sentence=params['ignore-first-sentence']
                      )
stbl = True if params['hc-type']=='stbl' else False

min_no_sentences = 10

results = []
too_short = []
for c in tqdm(ds_mixed_grouped.groupby('new_name')):
    responses = c[1]['response']
    lengths = c[1]['length']
    if len(responses) > min_no_sentences:
      pvals, comments = detectlm._get_pvals(responses, lengths)
      pvals = np.vstack(pvals).squeeze()
      mt = MultiTest(pvals, stbl=stbl)
      hc = mt.hc()[0]
      results.append(dict(id=c[0], HC=hc))
    else:
       too_short.append(c)

100%|██████████| 3304/3304 [00:01<00:00, 2527.09it/s]


In [264]:
crit_vals = pd.read_csv("HC_critvals.csv")

In [265]:
sig_level = 0.05
t0 = crit_vals[(crit_vals.n == min_length) & (crit_vals.alpha == sig_level)].q_alpha.values[0]
acc = np.mean(pd.DataFrame.from_dict(results)['HC'] > t0)

print(f"Model={model_name}, dataset={dataset_name}, epsilon={eps}, length={min_length}, sig_level={sig_level} --> detection rate {acc}")

Model=gpt2-xl, dataset=news, epsilon=0.1, length=100, sig_level=0.05 --> detection rate 0.3662227602905569
